# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901991


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:01<00:16,  1.64it/s]

Rendering models:  13%|█▎        | 4/30 [00:03<00:20,  1.25it/s]

Rendering models:  17%|█▋        | 5/30 [00:03<00:16,  1.55it/s]

Rendering models:  20%|██        | 6/30 [00:03<00:12,  1.87it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  2.87it/s]

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  3.21it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.30it/s]

Rendering models: 100%|██████████| 30/30 [00:13<00:00,  3.78it/s]

not-logged-in-9c310346c8df6cdafc65    0.016468
TestProfile24                         0.000090
THIRTYFATHOMGRAVE                     0.000258
ElisabethB                            0.000087
Jes8x                                 0.000202
R1chard                               0.000235
not-logged-in-ecf8362ef39037453165    0.000112
kayleebug2017                         0.000312
Nauter                                0.000216
gabrielweiss                          0.003145
Jatkins9752                           0.000097
not-logged-in-9f30f411df9d28d2c541    0.000209
acapirala                             0.000114
Lavadude                              0.037697
w7250369                              0.000205
Tessaje04                             0.144481
Fiona77h                              0.000211
slambier                              0.000088
sirtylerli                            0.000100
jmacd297                              0.000383
kgenere                               0.000124
Fortnitesucks

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())